# BIOMASS slices and frames

## Data take subdivision in slices

### Algorithms

The following algorithm describes how to determine the data take's subdivision in slices according to the L0 slice grid defined in the Production Model. 

```
Initialize slices_container
For each orbit in orbits_overlapping_data_take:
    For n = 1 to NUM_SLICES:
        slice_n = get_interval(n, orbit_start, orbit_stop, duration, initial_overlap, final_overlap, num_intervals)
        if slice_n overlaps data_take:
            updated_slice_start = max(data_take_start, slice_start)
            updated_slice_stop = min(data_take_stop, slice_stop)
            add updated_slice to slices_container
```
where:
- `data_take`: time interval corresponding to the data take start/stop times.
- `slices_container`: container object (initially empty)
- `orbit`: time interval defined by two consecutive ANX timestamps (note: data take can span across different orbits)
- `orbits_overlapping_data_take`: subset of orbits overlapping `data_take`
- `NUM_SLICES`: number of slices in an orbit
- `slice_n`: time interval of n<sup>th</sup> slice, with $n \in [1, \text{NUM_SLICES}]$
- `updated_slice`: time interval of n<sup>th</sup> slice, updated according to `data_take`.

```
function get_interval(n, orbit_start, orbit_stop, duration, initial_overlap, final_overlap, num_intervals):
    Verify: 0 < n <= num_intervals
    interval_start = orbit_start + (n - 1) * duration - initial_overlap
    If n == num_intervals:
        interval_stop = orbit_stop + final_overlap
    Else:
        interval_stop = orbit_start + n * duration + final_overlap
    Return (interval_start, interval_stop)
```

### Sample implementation

In [1]:
import pandas as pd


def get_interval(n: int, orbit_start: pd.Timestamp, orbit_stop: pd.Timestamp,
                 duration: float, initial_overlap: float, final_overlap: float,
                 num_intervals: int) -> pd.Interval:
    assert 0 < n <= num_intervals
    left = orbit_start + pd.to_timedelta((n - 1) * duration - initial_overlap, 's')
    if n == num_intervals:
        right = orbit_stop + pd.to_timedelta(final_overlap, 's')
    else:
        right = orbit_start + pd.to_timedelta(n * duration + final_overlap, 's')
    return pd.Interval(left, right, closed='both')
    

### Slice definitition

In [2]:
SLICE_GRID_DURATION = 95.01580774
SLICE_INITIAL_OVERLAP = 5
SLICE_FINAL_OVERLAP = 7
NUM_SLICES = 62
SLICE_DURATION = SLICE_GRID_DURATION + SLICE_INITIAL_OVERLAP + SLICE_FINAL_OVERLAP

### Example

In [3]:
# Define 3 ANXs identifying two orbits
anxs = pd.date_range('2021-01-01', periods=3, freq='5891s')
anxs

DatetimeIndex(['2021-01-01 00:00:00', '2021-01-01 01:38:11',
               '2021-01-01 03:16:22'],
              dtype='datetime64[ns]', freq='5891S')

In [4]:
# Note: frequency of 5891 seconds is slightly greater than theoretical orbital period:
NUM_SLICES * SLICE_GRID_DURATION

5890.98007988

In [5]:
# Define data take start/stop times (spanning across anxs[1]):
data_take = pd.Interval(pd.Timestamp('2021-01-01 01:30:00'), pd.Timestamp('2021-01-01 01:45:00'), closed='both')
data_take

Interval('2021-01-01 01:30:00', '2021-01-01 01:45:00', closed='both')

In [6]:
slices = []
for k in range(anxs.shape[0] - 1):    
    for n in range(NUM_SLICES):
        s = get_interval(n + 1, anxs[k], anxs[k + 1], SLICE_GRID_DURATION, SLICE_INITIAL_OVERLAP, SLICE_FINAL_OVERLAP, NUM_SLICES)
        if s.overlaps(data_take):
            start = max(data_take.left, s.left)
            stop = min(data_take.right, s.right)
            slices.append((n + 1, pd.Interval(start, stop, closed='both')))
            
slices = pd.DataFrame(
    [{'slice': s[0], 'start': s[1].left, 'stop': s[1].right}
     for s in slices]
)
slices['duration'] = (slices['stop'] - slices['start']).apply(lambda d: d.total_seconds())
slices

,slice,start,stop,duration
0,57,2021-01-01 01:30:00.000000000,2021-01-01 01:30:22.901041180,22.901041
1,58,2021-01-01 01:30:10.901041180,2021-01-01 01:31:57.916848920,107.015807
2,59,2021-01-01 01:31:45.916848920,2021-01-01 01:33:32.932656660,107.015807
3,60,2021-01-01 01:33:20.932656660,2021-01-01 01:35:07.948464400,107.015807
4,61,2021-01-01 01:34:55.948464400,2021-01-01 01:36:42.964272140,107.015807
5,62,2021-01-01 01:36:30.964272140,2021-01-01 01:38:18.000000000,107.035727
6,1,2021-01-01 01:38:06.000000000,2021-01-01 01:39:53.015807740,107.015807
7,2,2021-01-01 01:39:41.015807740,2021-01-01 01:41:28.031615480,107.015807
8,3,2021-01-01 01:41:16.031615480,2021-01-01 01:43:03.047423220,107.015807
9,4,2021-01-01 01:42:51.047423220,2021-01-01 01:44:38.063230960,107.015807


## Data take subdivision in frames

### Algorithms

The following algorithm describes how to determine the data take's subdivision in slices according to the L0 slice grid defined in the Production Model. 

```
Initialize frames_container
For each orbit in orbits_overlapping_data_take:
    For n = 1 to NUM_FRAMES:
        frame_n = get_interval(n, orbit_start, orbit_stop, duration, initial_overlap, final_overlap, num_intervals)
        if frame_n overlaps data_take:
            updated_frame_start = max(data_take_start, frame_start)
            updated_frame_stop = min(data_take_stop, frame_stop)
            add updated_frame to frames_container
```
where:
- `frames_container`: container object (initially empty)
- `NUM_FRAMES`: number of frames in an orbit
- `frame_n`: time interval of n<sup>th</sup> slice, with $n \in [1, \text{NUM_FRAMES}]$
- `updated_frame`: time interval of n<sup>th</sup> slice, updated according to `data_take`.


### Frame definitition

In [7]:
NUM_FRAMES_PER_SLICE = 5
FRAME_GRID_DURATION = SLICE_GRID_DURATION / NUM_FRAMES_PER_SLICE
FRAME_INITIAL_OVERLAP = 0
FRAME_FINAL_OVERLAP = 2
FRAME_DURATION = FRAME_GRID_DURATION + FRAME_INITIAL_OVERLAP + FRAME_FINAL_OVERLAP
NUM_FRAMES = NUM_SLICES * NUM_FRAMES_PER_SLICE

### Example

Same ANXs and data take as before.

In [8]:
frames = []
for k in range(anxs.shape[0] - 1):    
    for n in range(NUM_FRAMES):
        f = get_interval(n + 1, anxs[k], anxs[k + 1], FRAME_GRID_DURATION, FRAME_INITIAL_OVERLAP, FRAME_FINAL_OVERLAP, NUM_FRAMES)
        if f.overlaps(data_take):
            start = max(data_take.left, f.left)
            stop = min(data_take.right, f.right)
            frames.append((n + 1, pd.Interval(start, stop, closed='both')))
            
frames = pd.DataFrame(
    [{'frame': f[0], 'start': f[1].left, 'stop': f[1].right}
     for f in frames]
)
frames['duration'] = (frames['stop'] - frames['start']).apply(lambda d: d.total_seconds())
frames

,frame,start,stop,duration
0,285,2021-01-01 01:30:00.000000000,2021-01-01 01:30:17.901041180,17.901041
1,286,2021-01-01 01:30:15.901041180,2021-01-01 01:30:36.904202728,21.003161
2,287,2021-01-01 01:30:34.904202728,2021-01-01 01:30:55.907364276,21.003161
3,288,2021-01-01 01:30:53.907364276,2021-01-01 01:31:14.910525824,21.003161
4,289,2021-01-01 01:31:12.910525824,2021-01-01 01:31:33.913687372,21.003161
5,290,2021-01-01 01:31:31.913687372,2021-01-01 01:31:52.916848920,21.003161
6,291,2021-01-01 01:31:50.916848920,2021-01-01 01:32:11.920010468,21.003161
7,292,2021-01-01 01:32:09.920010468,2021-01-01 01:32:30.923172016,21.003161
8,293,2021-01-01 01:32:28.923172016,2021-01-01 01:32:49.926333564,21.003161
9,294,2021-01-01 01:32:47.926333564,2021-01-01 01:33:08.929495112,21.003161
